In [1]:
import pandas as pd
import numpy as np

# Charger le dataset
file_path = "accidentsVelo.csv"
df = pd.read_csv(file_path, low_memory=False)

# Suppression des accidents de gravité inférieure à 3
df_cleaned = df[df["grav"] >= 3]

# Colonnes ayant trop de valeurs manquantes (plus de 10%)
cols_to_drop = ["lartpc", "typevehicules", "manoeuvehicules", "numVehicules", "larrout"]
df_cleaned = df_cleaned.drop(columns=cols_to_drop)

# Conversion des colonnes mal formatées en float
cols_to_convert = ["long", "lat", "age"]
for col in cols_to_convert:
    df_cleaned[col] = df_cleaned[col].astype(str).str.replace(",", ".").astype(float)

# Remplissage des valeurs manquantes
fill_median = ["long", "lat", "age"]
fill_mode = ["nbv", "circ", "atm", "prof", "plan", "surf", "infra", "situ", "equipement", "obs", "obsm", "choc", "manv", "col", "trajet"]
for col in fill_median:
    df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)
for col in fill_mode:
    df_cleaned[col].fillna(df_cleaned[col].mode()[0], inplace=True)

# Extraction de l'heure pour l'analyse horaire
df_cleaned["heure"] = df_cleaned["hrmn"].astype(str).str.split(":").str[0].astype(int)

# Vérification finale des valeurs manquantes
df_cleaned.isnull().sum().sum()


0

In [ ]:
df_cleaned.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Distribution des accidents par gravité
plt.figure(figsize=(8, 5))
sns.countplot(x=df_cleaned["grav"], palette="coolwarm")
plt.title("Distribution des Accidents par Gravité")
plt.xlabel("Gravité de l'Accident")
plt.ylabel("Nombre d'Accidents")
plt.show()

# Répartition des accidents par heure
plt.figure(figsize=(10, 5))
sns.countplot(x=df_cleaned["heure"], palette="viridis")
plt.title("Répartition des Accidents par Heure")
plt.xlabel("Heure de la Journée")
plt.ylabel("Nombre d'Accidents")
plt.xticks(rotation=45)
plt.show()

# Carte statique des accidents
plt.figure(figsize=(10, 6))
plt.scatter(df_cleaned["long"], df_cleaned["lat"], alpha=0.3, s=1, color="red")
plt.title("Répartition géographique des accidents")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


In [3]:
# Correction des valeurs contenant des "/"
for col in ["equipement", "choc", "manv"]:
    df_cleaned[col] = df_cleaned[col].astype(str).str.split("/").str[0]
    df_cleaned[col] = df_cleaned[col].astype(float)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Sélection des features et de la target
features = ["heure", "long", "lat", "nbv", "circ", "atm", "prof", "plan", "surf", "infra", "situ", "equipement", "choc", "manv"]
X = df_cleaned[features]
y = df_cleaned["grav"]  # Gravité de l'accident comme variable cible

# Correction des valeurs contenant des "/"
for col in ["equipement", "choc", "manv"]:
    df_cleaned[col] = df_cleaned[col].astype(str).str.split("/").str[0]
    df_cleaned[col] = df_cleaned[col].astype(float)

# Séparation en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisation des features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialisation des modèles
models = {
    "Régression Logistique": LogisticRegression(max_iter=500),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

# Entraînement et évaluation des modèles
results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    print(f"Modèle : {name} - Accuracy : {accuracy:.4f}")
    print(classification_report(y_test, y_pred))


In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(x=list(results.keys()), y=list(results.values()), palette="magma")
plt.title("Performance des modèles (Accuracy)")
plt.xlabel("Modèle")
plt.ylabel("Score d'Accuracy")
plt.show()


In [7]:
# Sélection du meilleur modèle
best_model = models["Random Forest"]
df_cleaned["grav_pred"] = best_model.predict(scaler.transform(df_cleaned[features]))


In [ ]:
%pip install folium
import folium
from folium.plugins import HeatMap

# Création de la carte interactive
m = folium.Map(location=[df_cleaned["lat"].mean(), df_cleaned["long"].mean()], zoom_start=6)

# Ajout des points sur la carte avec couleur selon la gravité prédite
for _, row in df_cleaned.iterrows():
    color = "red" if row["grav_pred"] == 4 else "orange"
    folium.CircleMarker(
        location=[row["lat"], row["long"]],
        radius=2,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
    ).add_to(m)

# Ajout d'une Heatmap pour voir les zones critiques
heat_data = df_cleaned[["lat", "long"]].values.tolist()
HeatMap(heat_data, radius=10).add_to(m)

# Affichage de la carte interactive
m


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Création d'un boxplot pour visualiser la distribution de la gravité en fonction de l'heure
plt.figure(figsize=(10, 5))
sns.boxplot(x=df_cleaned["heure"], y=df_cleaned["grav"], palette="coolwarm")
plt.title("Impact de l'heure sur la Gravité des Accidents")
plt.xlabel("Heure de la Journée")
plt.ylabel("Gravité de l'Accident")
plt.show()
# Affichage des fréquences d'accidents pour chaque heure
hour_counts = df_cleaned['heure'].value_counts().sort_index()
print(hour_counts)

# Visualisation des horaires avec un barplot
plt.figure(figsize=(10, 5))
sns.barplot(x=hour_counts.index, y=hour_counts.values, palette="viridis")
plt.title("Nombre d'Accidents par Heure")
plt.xlabel("Heure de la Journée")
plt.ylabel("Nombre d'Accidents")
plt.xticks(rotation=45)  # Rotation des étiquettes pour une meilleure lisibilité
plt.show()

In [ ]:
# Regrouper les accidents par département et calculer la gravité moyenne
top_departments = df_cleaned.groupby("dep")["grav"].mean().sort_values(ascending=False).head(10)

# Création du graphique
plt.figure(figsize=(12, 6))
sns.barplot(x=top_departments.index, y=top_departments.values, palette="Reds")
plt.title("Départements avec les Accidents les Plus Graves")
plt.xlabel("Département")
plt.ylabel("Gravité Moyenne")
plt.show()


In [ ]:
# Création d'un boxplot pour voir la répartition de l'âge selon la gravité
plt.figure(figsize=(8, 5))
sns.boxplot(x=df_cleaned["grav"], y=df_cleaned["age"], palette="viridis")
plt.title("Âge des Conducteurs en Fonction de la Gravité de l'Accident")
plt.xlabel("Gravité de l'Accident")
plt.ylabel("Âge du Conducteur")
plt.show()


In [ ]:
# Création de la matrice de corrélation
plt.figure(figsize=(12, 8))
correlation_matrix = df_cleaned[["grav", "heure", "long", "lat", "age", "nbv", "circ", "atm", "prof", "plan", "surf", "infra", "situ", "equipement", "choc", "manv"]].corr()

# Affichage sous forme de heatmap
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrice de Corrélation des Variables")
plt.show()


In [ ]:
# Création d'un countplot pour voir l'influence des conditions météo sur la gravité
plt.figure(figsize=(10, 5))
sns.countplot(x=df_cleaned["atm"], hue=df_cleaned["grav"], palette="coolwarm")
plt.title("Impact des Conditions Atmosphériques sur la Gravité des Accidents")
plt.xlabel("Conditions Atmosphériques")
plt.ylabel("Nombre d'Accidents")
plt.legend(title="Gravité")
plt.show()


In [ ]:
# Création d'un countplot pour voir l'influence du type de choc sur la gravité
plt.figure(figsize=(10, 5))
sns.countplot(x=df_cleaned["choc"], hue=df_cleaned["grav"], palette="magma")
plt.title("Impact du Type de Choc sur la Gravité des Accidents")
plt.xlabel("Type de Choc")
plt.ylabel("Nombre d'Accidents")
plt.legend(title="Gravité")
plt.show()


In [ ]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
import joblib

# Chargement du dataset
file_path = "accidentsVelo.csv"
df = pd.read_csv(file_path, low_memory=False)

# Suppression des accidents de gravité inférieure à 3
df_cleaned = df[df["grav"] >= 3]

# Suppression des colonnes avec trop de valeurs manquantes
cols_to_drop = ["lartpc", "typevehicules", "manoeuvehicules", "numVehicules", "larrout"]
df_cleaned = df_cleaned.drop(columns=cols_to_drop)

# Conversion des colonnes en float
cols_to_convert = ["long", "lat", "age"]
for col in cols_to_convert:
    df_cleaned[col] = df_cleaned[col].astype(str).str.replace(",", ".").astype(float)

# Remplissage des valeurs manquantes
fill_median = ["long", "lat", "age"]
fill_mode = ["nbv", "circ", "atm", "prof", "plan", "surf", "infra", "situ", "equipement", "obs", "obsm", "choc", "manv", "col", "trajet"]
for col in fill_median:
    df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)
for col in fill_mode:
    df_cleaned[col].fillna(df_cleaned[col].mode()[0], inplace=True)

# Extraction de l'heure de l'accident
df_cleaned["heure"] = df_cleaned["hrmn"].astype(str).str.split(":").str[0].astype(int)

# Correction des valeurs contenant "/"
for col in ["equipement", "choc", "manv"]:
    df_cleaned[col] = df_cleaned[col].astype(str).str.split("/").str[0].astype(float)

# Sélection des features et variable cible
features = ["heure", "long", "lat", "nbv", "circ", "atm", "prof", "plan", "surf", "infra", "situ", "equipement", "choc", "manv"]
X = df_cleaned[features]
y = df_cleaned["grav"]

# Séparation en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisation des features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Grilles de recherche pour l'optimisation
param_grid_rf = {"n_estimators": [50, 100], "max_depth": [None, 10]}
param_grid_knn = {"n_neighbors": [3, 5]}
param_grid_lr = {"C": [0.1, 1]}

models_params = {
    "Random Forest": (RandomForestClassifier(random_state=42), param_grid_rf),
    "k-NN": (KNeighborsClassifier(), param_grid_knn),
    "Régression Logistique": (LogisticRegression(max_iter=500), param_grid_lr)
}

# Optimisation et évaluation des modèles
best_models = {}
model_scores = {}

for name, (model, params) in models_params.items():
    grid_search = GridSearchCV(model, params, cv=3, scoring="accuracy", n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)
    
    best_model = grid_search.best_estimator_
    best_models[name] = best_model
    
    y_pred = best_model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    model_scores[name] = accuracy
    
    print(f"Meilleur modèle {name} - Accuracy : {accuracy:.4f}")
    print(f"Meilleurs paramètres : {grid_search.best_params_}")
    
    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Prédictions")
    plt.ylabel("Vraies valeurs")
    plt.title(f"Matrice de Confusion - {name}")
    plt.show()

# Sauvegarde du meilleur modèle
best_model_name = max(model_scores, key=model_scores.get)
joblib.dump(best_models[best_model_name], "meilleur_modele.pkl")
print(f"Le modèle sauvegardé est : {best_model_name}")
